<a href="https://colab.research.google.com/github/uyan21/deepLearningStudy/blob/main/advanced_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# advaced mnist
# Tensorflow 사이트에 mnist 전문가용 버전 코드 분석

#1.tensorflow 임포트한다 (tf 계열 함수는 쓸모가 많으므로 일단 임포트하는거임)
#2.레이어 임포트한다 뒤에는 레이어 종류 임포트 시킨다
#3.Sequential 전문가용 버전은 시퀀셜 모델 안쓴다
#4.one-hot encoding 함수용 함수
#예) 10진수의 [8] 데이터라면 ->[0,0,0,0,0,0,0,0,1,0]
#    8진수의 [5] 데이터라면 ->[0,0,0,0,0,1,0,0]
#아웃풋 레이어는 이렇게 만들어놔야함 불문율

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model
from keras.utils import to_categorical

#mnist 데이터 받는다
#255로 나누는것은 0~1사이 값으로 만드는 일명 정규화(normalization), 맵핑이라고도 함

In [ ]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [ ]:
print(x_train.shape)

(60000, 28, 28, 1)


#shuffle 10000은 섞는데 쓸 버퍼용량이 10000이라는 것(중요하지 않음)
#batch가 32(32개씩 묶는 것임)
#묶는 이유는 한꺼번에 32개 넣고 채점하고 넣고 채점 뒤에 함수 그렇게 해놈

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

#여기서는 레이어 모델을 클래스 상속을 하여 레이어를 쌓는다
#인풋디멘션과 아웃풋 디멘션이 데이터 집어넣는 
#즉시 알아서 첫 데이터 크기에 맞춰진다 -> 따로 설정할 필요 없으므로 편함
#call 함수 이름 그대로 써야함 클래스 상속이기 때문임
#(슈퍼클래스에 call이 이미 있어서 그대로 상속해서 
#내부 채우는 것 이것을 오버라이딩이라 한다)

In [ ]:
class MyModel(Model):
  def __init__(self):
    super(MyModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')
    
  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

model = MyModel()


#loss_object에 할당된 매서드가 뭐냐하면 loss_object(정답,예측 값)
#넣고 일치율 계산용

#optimizer 최적화 함수 종류 중 대표적인 adam (이거는 피드백 하는 거임, 말하면 김)


In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()
print(loss_object)
print(optimizer)

#1,2줄 학습시 손실률과 1 batch 끝나고 정확성(채점할때 쓴다)

#train_step 함수 정의
#7줄 모델에서 예측 값 뱉고
#8줄에서 정답하고 비교한다

#그 밑에 gradientTape가 뭐냐면 저거는 미분용 함수인데 경사하강법을 쓰기 위해서 
#(경사하강법으로 최적점을 찾아간다, 말하면 김)

In [ ]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

#테스트데이터로 학습모델 테스트할때 쓰는 함수 
#위에 train_step에서 gradientTape 빠짐(피드 백,학습 할 필요 없으니까)

In [ ]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)
  return test_accuracy.result()

#epochs 반복수 5
#그 밑에 for문으로 batch 설정크기 (32개씩) 짤라서 넣는다

In [ ]:
EPOCHS=5
for epoch in range(EPOCHS):
  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
  print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.13814786076545715, 정확도: 95.88500213623047, 테스트 손실: 0.054303914308547974, 테스트 정확도: 98.29000091552734
에포크: 2, 손실: 0.09018687158823013, 정확도: 97.28166198730469, 테스트 손실: 0.057121116667985916, 테스트 정확도: 98.18000030517578
에포크: 3, 손실: 0.06752774864435196, 정확도: 97.96888732910156, 테스트 손실: 0.05558275803923607, 테스트 정확도: 98.22999572753906
에포크: 4, 손실: 0.053904805332422256, 정확도: 98.37458038330078, 테스트 손실: 0.0565926656126976, 테스트 정확도: 98.26249694824219
에포크: 5, 손실: 0.04475361481308937, 정확도: 98.64633178710938, 테스트 손실: 0.05684742331504822, 테스트 정확도: 98.3219985961914


#이렇게 어려운 버전을 왜 쓰냐면
#쉬운버전은 시퀀셜(일렬 레이어 모델) 밖에 설계가 안된다
#이것보다 복잡한 레이어 모델을 만들려면 Sequential() 이거부터 시작해서 
#model.add(레이어) 이런거 가지고는 안됨
#(예 레이어가 상황에 따라 또 레이어를 만드는 유전 알고리즘,
#분류후 각기 다른 레이어에 넣는 모델)